# 🎨 Clothing Classifier Training Notebook

This notebook trains a clothing classification model using **MobileNetV2** transfer learning.

**Model Architecture:**
- Base: MobileNetV2 (ImageNet pretrained)
- Custom top: Dense layers with dropout
- Output: 6 clothing categories

**Training Strategy:**
- Phase 1: Frozen base (15 epochs)
- Phase 2: Fine-tuning last 20 layers (15 epochs)

**Expected Performance:** 95%+ accuracy

## 1️⃣ Setup & Imports

In [1]:
# Suppress TensorFlow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from pathlib import Path
from datetime import datetime
import json

# Visualization imports
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ All imports successful!")
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {tf.keras.__version__}")

✅ All imports successful!
TensorFlow version: 2.10.0
Keras version: 2.10.0


In [ ]:
# Check GPU availability
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"🎮 GPU detected: {gpus[0].name}")
    print(f"   Memory growth enabled")
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print(f"   ✅ GPU will be used for training (6x faster)")
else:
    print("⚠️  No GPU detected, using CPU")
    print("   Training will be slower but still functional")
    print()
    print("📌 You have RTX 3050 GPU but TensorFlow can't detect it.")
    print("   This is NORMAL - you only have Python GPU packages, not CUDA Toolkit.")
    print()
    print("✅ OPTION 1: Continue with CPU (Recommended for now)")
    print("   - Training will work perfectly fine")
    print("   - Just takes longer (~2-3 hours vs 30-40 minutes)")
    print("   - No installation needed")
    print()
    print("🚀 OPTION 2: Enable GPU (Optional - for 6x speedup)")
    print("   Step 1: Download CUDA Toolkit 12.6")
    print("           https://developer.nvidia.com/cuda-downloads")
    print("   Step 2: Install with default settings")
    print("   Step 3: Restart Jupyter notebook")
    print("   Step 4: Re-run this cell - GPU will be detected")
    print()
    print("💡 For this tutorial, CPU mode is fine. You can enable GPU later.")

⚠️  No GPU detected, using CPU
   Training will be slower but still functional


In [ ]:
# Verify compute device and estimate training time
device_name = "GPU" if tf.config.list_physical_devices('GPU') else "CPU"
print(f"\n{'='*60}")
print(f"🖥️  COMPUTE DEVICE: {device_name}")
print(f"{'='*60}")

if device_name == "CPU":
    print("⏱️  Estimated training time:")
    print("   - Phase 1 (15 epochs): ~60-90 minutes")
    print("   - Phase 2 (15 epochs): ~60-90 minutes")
    print("   - Total: ~2-3 hours")
    print()
    print("💡 TIP: You can reduce epochs for faster testing:")
    print("   EPOCHS1 = 5  # Instead of 15")
    print("   EPOCHS2 = 5  # Instead of 15")
else:
    print("⏱️  Estimated training time:")
    print("   - Phase 1 (15 epochs): ~15-20 minutes")
    print("   - Phase 2 (15 epochs): ~15-20 minutes")
    print("   - Total: ~30-40 minutes")

print(f"{'='*60}\n")
print("✅ Ready to start training!")

## 2️⃣ Configuration & Paths

In [ ]:
# Update paths for Windows
ROOT = Path(r'C:\Users\Prachi\Desktop\qq\AIProject')
DATA = ROOT / 'data'
PROCESSED = DATA / 'processed'
RAW = DATA / 'raw'
MODELS = ROOT / 'models' / 'saved_models'
MODELS.mkdir(parents=True, exist_ok=True)

# Training configuration
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS1 = 15  # Phase 1: Frozen base
EPOCHS2 = 15  # Phase 2: Fine-tuning

print("=" * 80)
print("🚀 CLOTHING CLASSIFIER TRAINING")
print("=" * 80)
print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Data directory: {PROCESSED}")
print(f"Models directory: {MODELS}")
print(f"Image size: {IMG_SIZE}x{IMG_SIZE}")
print(f"Batch size: {BATCH_SIZE}")

## 3️⃣ Load & Explore Data

In [ ]:
# Load datasets
print("📂 Loading datasets...")
train_df = pd.read_csv(PROCESSED / 'train.csv')
val_df = pd.read_csv(PROCESSED / 'val.csv')
test_df = pd.read_csv(PROCESSED / 'test.csv')

# Load label mapping
with open(PROCESSED / 'label_mapping.json') as f:
    label_mapping = json.load(f)

num_classes = len(label_mapping)

print(f"\n📊 Dataset Statistics:")
print(f"   Train samples: {len(train_df):,}")
print(f"   Validation samples: {len(val_df):,}")
print(f"   Test samples: {len(test_df):,}")
print(f"   Total classes: {num_classes}")
print(f"\n📋 Classes: {list(label_mapping.keys())}")

In [ ]:
# Visualize class distribution
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Training set distribution
train_counts = train_df['main_category'].value_counts()
axes[0].bar(train_counts.index, train_counts.values, color='skyblue')
axes[0].set_title('Training Set Class Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Category')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=45)
for i, v in enumerate(train_counts.values):
    axes[0].text(i, v + 100, str(v), ha='center', fontweight='bold')

# Validation set distribution
val_counts = val_df['main_category'].value_counts()
axes[1].bar(val_counts.index, val_counts.values, color='lightcoral')
axes[1].set_title('Validation Set Class Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Category')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=45)
for i, v in enumerate(val_counts.values):
    axes[1].text(i, v + 20, str(v), ha='center', fontweight='bold')

# Test set distribution
test_counts = test_df['main_category'].value_counts()
axes[2].bar(test_counts.index, test_counts.values, color='lightgreen')
axes[2].set_title('Test Set Class Distribution', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Category')
axes[2].set_ylabel('Count')
axes[2].tick_params(axis='x', rotation=45)
for i, v in enumerate(test_counts.values):
    axes[2].text(i, v + 20, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✅ Class distributions visualized")

## 4️⃣ Create TensorFlow Datasets

In [ ]:
def make_dataset(df, training=True):
    """
    Create TensorFlow dataset from DataFrame.
    
    Args:
        df: DataFrame with 'id' and 'category_label' columns
        training: If True, apply data augmentation
    
    Returns:
        tf.data.Dataset
    """
    def load_img(path, label):
        # Read and decode image
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
        img = tf.cast(img, tf.float32) / 255.0
        
        # Data augmentation for training
        if training:
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_brightness(img, 0.2)
            img = tf.image.random_contrast(img, 0.8, 1.2)
        
        return img, label
    
    # Create file paths
    paths = [str(RAW / 'images' / f"{row['id']}.jpg") for _, row in df.iterrows()]
    labels = df['category_label'].values
    
    # Create dataset
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(load_img, num_parallel_calls=tf.data.AUTOTUNE)
    
    if training:
        ds = ds.shuffle(buffer_size=1000)
    
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    
    return ds

print("📊 Creating TensorFlow datasets...")
train_ds = make_dataset(train_df, training=True)
val_ds = make_dataset(val_df, training=False)
test_ds = make_dataset(test_df, training=False)

print("✅ Datasets created successfully!")
print(f"   Training batches: ~{len(train_df) // BATCH_SIZE}")
print(f"   Validation batches: ~{len(val_df) // BATCH_SIZE}")
print(f"   Test batches: ~{len(test_df) // BATCH_SIZE}")

In [ ]:
# Visualize sample images from training set
sample_batch = next(iter(train_ds))
sample_images, sample_labels = sample_batch

# Reverse label mapping
reverse_mapping = {v: k for k, v in label_mapping.items()}

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i in range(8):
    axes[i].imshow(sample_images[i].numpy())
    label_name = reverse_mapping[int(sample_labels[i])]
    axes[i].set_title(f'{label_name}', fontsize=12, fontweight='bold')
    axes[i].axis('off')

plt.suptitle('Sample Training Images with Augmentation', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("✅ Sample images visualized")

## 5️⃣ Build Model Architecture

In [ ]:
print("🏗️  Building model architecture...")

# Load pre-trained MobileNetV2
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.trainable = False  # Freeze for Phase 1

# Build custom classifier on top
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(num_classes, activation='softmax')
], name='ClothingClassifier')

print(f"✅ Model built successfully!")
print(f"   Total parameters: {model.count_params():,}")
print(f"   Trainable parameters: {sum([tf.size(w).numpy() for w in model.trainable_weights]):,}")
print(f"   Non-trainable parameters: {sum([tf.size(w).numpy() for w in model.non_trainable_weights]):,}")

In [ ]:
# Display model architecture
model.summary()

In [ ]:
# Visualize model architecture
from tensorflow.keras.utils import plot_model

plot_model(
    model,
    to_file='clothing_classifier_architecture.png',
    show_shapes=True,
    show_layer_names=True,
    rankdir='TB',
    expand_nested=False,
    dpi=96
)

print("✅ Model architecture diagram saved!")

## 6️⃣ Phase 1: Train with Frozen Base

In [ ]:
print("=" * 80)
print("PHASE 1: Training with Frozen Base")
print("=" * 80)
print("Only training custom top layers while base model is frozen")
print(f"Learning rate: 0.001")
print(f"Epochs: {EPOCHS1}")

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
callbacks_phase1 = [
    tf.keras.callbacks.ModelCheckpoint(
        str(MODELS / 'classifier_phase1_best.keras'),
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    )
]

print("\n🚀 Starting Phase 1 training...\n")

In [ ]:
# Train Phase 1
history_phase1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS1,
    callbacks=callbacks_phase1,
    verbose=1
)

print("\n✅ Phase 1 training complete!")

In [ ]:
# Plot Phase 1 training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy plot
axes[0].plot(history_phase1.history['accuracy'], label='Training', linewidth=2, marker='o')
axes[0].plot(history_phase1.history['val_accuracy'], label='Validation', linewidth=2, marker='s')
axes[0].set_title('Phase 1: Model Accuracy', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss plot
axes[1].plot(history_phase1.history['loss'], label='Training', linewidth=2, marker='o')
axes[1].plot(history_phase1.history['val_loss'], label='Validation', linewidth=2, marker='s')
axes[1].set_title('Phase 1: Model Loss', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Phase 1 Best Validation Accuracy: {max(history_phase1.history['val_accuracy']):.4f}")

## 7️⃣ Phase 2: Fine-tuning with Unfrozen Layers

In [ ]:
print("=" * 80)
print("PHASE 2: Fine-tuning")
print("=" * 80)
print("Unfreezing last 20 layers of base model for fine-tuning")

# Unfreeze base model
base_model.trainable = True

# Freeze all layers except last 20
for layer in base_model.layers[:-20]:
    layer.trainable = False

trainable_layers = sum([layer.trainable for layer in base_model.layers])
print(f"\nTrainable layers in base model: {trainable_layers}/{len(base_model.layers)}")

# Recompile with lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),  # Much lower LR
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print(f"Learning rate: 0.00001 (10x lower than Phase 1)")
print(f"Epochs: {EPOCHS2}")

# Callbacks for Phase 2
callbacks_phase2 = [
    tf.keras.callbacks.ModelCheckpoint(
        str(MODELS / 'classifier_phase2_best.keras'),
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-8,
        verbose=1
    )
]

print("\n🚀 Starting Phase 2 training...\n")

In [ ]:
# Train Phase 2
history_phase2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS2,
    callbacks=callbacks_phase2,
    verbose=1
)

print("\n✅ Phase 2 training complete!")

In [ ]:
# Plot Phase 2 training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy plot
axes[0].plot(history_phase2.history['accuracy'], label='Training', linewidth=2, marker='o', color='green')
axes[0].plot(history_phase2.history['val_accuracy'], label='Validation', linewidth=2, marker='s', color='orange')
axes[0].set_title('Phase 2: Model Accuracy (Fine-tuning)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss plot
axes[1].plot(history_phase2.history['loss'], label='Training', linewidth=2, marker='o', color='green')
axes[1].plot(history_phase2.history['val_loss'], label='Validation', linewidth=2, marker='s', color='orange')
axes[1].set_title('Phase 2: Model Loss (Fine-tuning)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Phase 2 Best Validation Accuracy: {max(history_phase2.history['val_accuracy']):.4f}")

In [ ]:
# Compare both phases
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Combined accuracy
phase1_acc = history_phase1.history['val_accuracy']
phase2_acc = history_phase2.history['val_accuracy']
combined_acc = phase1_acc + phase2_acc

axes[0].plot(range(1, len(combined_acc) + 1), combined_acc, linewidth=2, marker='o', color='purple')
axes[0].axvline(x=len(phase1_acc), color='red', linestyle='--', linewidth=2, label='Phase 1 → Phase 2')
axes[0].set_title('Combined Training: Validation Accuracy', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Validation Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Combined loss
phase1_loss = history_phase1.history['val_loss']
phase2_loss = history_phase2.history['val_loss']
combined_loss = phase1_loss + phase2_loss

axes[1].plot(range(1, len(combined_loss) + 1), combined_loss, linewidth=2, marker='s', color='brown')
axes[1].axvline(x=len(phase1_loss), color='red', linestyle='--', linewidth=2, label='Phase 1 → Phase 2')
axes[1].set_title('Combined Training: Validation Loss', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Validation Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8️⃣ Evaluate on Test Set

In [ ]:
print("=" * 80)
print("EVALUATION ON TEST SET")
print("=" * 80)

# Evaluate
test_results = model.evaluate(test_ds, return_dict=True, verbose=1)

print(f"\n📊 Test Results:")
print(f"   Loss: {test_results['loss']:.4f}")
print(f"   Accuracy: {test_results['accuracy']*100:.2f}%")

In [ ]:
# Get predictions for confusion matrix
print("\n📊 Generating predictions for test set...")
y_true = []
y_pred = []

for images, labels in test_ds:
    predictions = model.predict(images, verbose=0)
    pred_classes = np.argmax(predictions, axis=1)
    
    y_true.extend(labels.numpy())
    y_pred.extend(pred_classes)

y_true = np.array(y_true)
y_pred = np.array(y_pred)

print(f"✅ Generated {len(y_true)} predictions")

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
class_names = list(label_mapping.keys())

plt.figure(figsize=(12, 10))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=class_names,
    yticklabels=class_names,
    cbar_kws={'label': 'Count'}
)
plt.title('Confusion Matrix - Test Set', fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("✅ Confusion matrix visualized")

In [ ]:
# Classification Report
report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
report_df = pd.DataFrame(report).transpose()

print("\n📋 Classification Report:")
print(report_df.to_string())

# Visualize per-class metrics
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics_df = report_df.iloc[:-3, :3]  # Exclude avg rows

# Precision
axes[0].barh(metrics_df.index, metrics_df['precision'], color='skyblue')
axes[0].set_xlabel('Precision', fontweight='bold')
axes[0].set_title('Per-Class Precision', fontsize=14, fontweight='bold')
axes[0].set_xlim([0, 1])
axes[0].grid(axis='x', alpha=0.3)

# Recall
axes[1].barh(metrics_df.index, metrics_df['recall'], color='lightcoral')
axes[1].set_xlabel('Recall', fontweight='bold')
axes[1].set_title('Per-Class Recall', fontsize=14, fontweight='bold')
axes[1].set_xlim([0, 1])
axes[1].grid(axis='x', alpha=0.3)

# F1-Score
axes[2].barh(metrics_df.index, metrics_df['f1-score'], color='lightgreen')
axes[2].set_xlabel('F1-Score', fontweight='bold')
axes[2].set_title('Per-Class F1-Score', fontsize=14, fontweight='bold')
axes[2].set_xlim([0, 1])
axes[2].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## 9️⃣ Save Final Model

In [ ]:
# Save final model
final_model_path = MODELS / 'clothing_classifier.keras'
model.save(final_model_path)

print(f"\n💾 Final model saved: {final_model_path}")
print(f"   Model size: {final_model_path.stat().st_size / (1024**2):.2f} MB")

# Save training history
history_dict = {
    'phase1': {
        'loss': [float(x) for x in history_phase1.history['loss']],
        'accuracy': [float(x) for x in history_phase1.history['accuracy']],
        'val_loss': [float(x) for x in history_phase1.history['val_loss']],
        'val_accuracy': [float(x) for x in history_phase1.history['val_accuracy']]
    },
    'phase2': {
        'loss': [float(x) for x in history_phase2.history['loss']],
        'accuracy': [float(x) for x in history_phase2.history['accuracy']],
        'val_loss': [float(x) for x in history_phase2.history['val_loss']],
        'val_accuracy': [float(x) for x in history_phase2.history['val_accuracy']]
    },
    'test': {
        'loss': float(test_results['loss']),
        'accuracy': float(test_results['accuracy'])
    }
}

history_path = MODELS / 'history.json'
with open(history_path, 'w') as f:
    json.dump(history_dict, f, indent=2)

print(f"✅ Training history saved: {history_path}")

## 🎉 Training Complete!

### Summary:
- ✅ Model trained with 2-phase approach
- ✅ Final test accuracy: **{:.2f}%**
- ✅ Model saved and ready for inference
- ✅ Complete training history saved

### Next Steps:
1. Use the model in `backend/main.py` for API inference
2. Test the model with real clothing images
3. Fine-tune further if needed with more data
4. Deploy to production environment

In [ ]:
print("=" * 80)
print("🎉 TRAINING COMPLETE!")
print("=" * 80)
print(f"Finished: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\nFinal Test Accuracy: {test_results['accuracy']*100:.2f}%")
print(f"Model saved to: {final_model_path}")